In [50]:
# -*- coding: utf-8 -*-
# ======================================================================
# CDR — Analysis & Preregistration Helpers (REMOTE-ONLY via WebDAV)
# Requires: cdr_keep_after_attn.parquet (written to OUT_DIR by the build step)
# Optional deps: pandas numpy pyarrow statsmodels
# ======================================================================

import re
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf

# ---- WebDAV I/O helpers
from rd_io import (
    DATA_DIR, OUT_DIR, rd_join, rd_write_markdown, rd_write_csv_df, rd_read_parquet_df, rd_read_text
)

# ---------------------------------------------------------------------
# Remote I/O config
# ---------------------------------------------------------------------
IN_PARQUET = rd_join(OUT_DIR, "cdr_keep_after_attn.parquet")  # from the build step

def save_markdown(name: str, content: str):
    """Write a Markdown file to the remote OUT_DIR and log a short note."""
    remote_path = rd_join(OUT_DIR, name)
    rd_write_markdown(content, remote_path)
    print(f"[written] {remote_path}")
    return remote_path

# -----------------------------
# Load (REMOTE)
# -----------------------------
df = rd_read_parquet_df(IN_PARQUET)

# -----------------------------
# STRICT age-band mapping from exact Dutch labels in "Age "
# -----------------------------
if "Age " in df.columns:
    age_map = {
        "18 - 24 jaar": "18–24",
        "25 - 34 jaar": "25–34",
        "35 - 44 jaar": "35–44",
        "45 - 54 jaar": "45–54",
        "55 - 65 jaar": "55–65",
    }

    def _to_band(x):
        if pd.isna(x):
            return np.nan
        s = str(x).strip()
        if s in age_map:
            return age_map[s]
        s2 = s.replace(" ", "").replace("–", "-")
        for k, v in age_map.items():
            if s2.startswith(k.replace(" ", "").replace("–", "-")):
                return v
        return np.nan

    bands = df["Age "].map(_to_band)
    allowed = ["18–24", "25–34", "35–44", "45–54", "55–65"]
    df["dem_age_band"] = pd.Categorical(bands, categories=allowed, ordered=True)
    if "dem_age_band_raw" not in df.columns:
        df["dem_age_band_raw"] = df["Age "]
    df = df[df["dem_age_band"].notna()].copy()

# -----------------------------
# Helpers (formatting + stats)
# -----------------------------
def cronbach_alpha(df_in, cols):
    cols = [c for c in cols if c in df_in.columns]
    if len(cols) < 2:
        return np.nan, 0
    sub = df_in[cols].apply(pd.to_numeric, errors="coerce").dropna()
    n, k = sub.shape
    if n < 2 or k < 2:
        return np.nan, n
    var_sum = sub.var(ddof=1).sum()
    total_var = sub.sum(axis=1).var(ddof=1)
    if total_var <= 0:
        return np.nan, n
    alpha = (k/(k-1))*(1 - var_sum/total_var)
    return float(alpha), n

def _fmt_val(v):
    if pd.isna(v):
        return ""
    if isinstance(v, (int, np.integer)):
        return f"{int(v):,}".replace(",", " ")
    if isinstance(v, (float, np.floating)):
        if float(v).is_integer():
            return f"{int(v):,}".replace(",", " ")
        s = f"{v:.3f}".rstrip("0").rstrip(".")
        return s
    return str(v)

def md_table(df_md):
    """Pretty Markdown table: left-align text, right-align numbers, neat formatting."""
    df_md = df_md.copy()
    for c in df_md.columns:
        df_md[c] = df_md[c].map(_fmt_val)
    cols = list(df_md.columns)
    lines = []
    lines.append("| " + " | ".join(cols) + " |")
    aligns = []
    for i, c in enumerate(cols):
        try:
            if pd.to_numeric(df_md[c].str.replace(" ", ""), errors="coerce").notna().mean() > 0.6 and i != 0:
                aligns.append("--:")
            else:
                aligns.append(":--")
        except Exception:
            aligns.append(":--")
    lines.append("|" + "|".join(aligns) + "|")
    for _, r in df_md.iterrows():
        lines.append("| " + " | ".join("" if pd.isna(v) else str(v) for v in r.values) + " |")
    return "\n".join(lines)

def present(cols):
    return [c for c in cols if c in df.columns]

def vc_pct(series, topn=20):
    """Value counts with %; if categorical, keep category order."""
    if hasattr(series, "cat"):
        vc = series.value_counts(dropna=False, sort=False)  # preserve order
    else:
        vc = series.value_counts(dropna=False)
    pct = (vc / vc.sum() * 100).round(1)
    out = pd.DataFrame({"n": vc, "%": pct})
    return out.head(topn)

def _repr_cols(n=60):
    return [repr(c) for c in list(df.columns)[:n]]

def _find_col(df_in, *cands):
    for c in cands:
        if c in df_in.columns: return c
        c2 = c.strip()
        if c2 in df_in.columns: return c2
    lower_map = {str(k).strip().lower(): k for k in df_in.columns}
    for c in cands:
        key = c.strip().lower()
        if key in lower_map: return lower_map[key]
    for c in cands:
        key = c.strip().lower()
        for k, orig in lower_map.items():
            if key in k:
                return orig
    return None

# -----------------------------
# Ensure other demogs exist (gender/education/employment/LR)
# -----------------------------
def ensure_other_demogs():
    raw_gend = _find_col(df, "Gender", "Gender ", "Geslacht")
    raw_edu  = _find_col(df, "Education", "Education ", "Opleiding")
    raw_work = _find_col(df, "Work_status", "Work status", "Werkstatus")
    raw_pol  = _find_col(df, "Pol_orientation", "Pol orientation", "Politieke", "Politiek", "L–R", "LR", "LeftRight")

    if "dem_gender" not in df.columns and raw_gend is not None:
        df["dem_gender"] = df[raw_gend].astype("category")
    if "dem_education" not in df.columns and raw_edu is not None:
        df["dem_education"] = df[raw_edu].astype("category")
    if "dem_employment" not in df.columns and raw_work is not None:
        df["dem_employment"] = df[raw_work].astype("category")

    if "dem_political_lr" not in df.columns and raw_pol is not None:
        s = pd.to_numeric(df[raw_pol], errors="coerce")
        if s.notna().sum() >= 10:
            df["dem_political_lr"] = s
        else:
            def _first_int(x):
                if pd.isna(x): return np.nan
                m = re.search(r'[-+]?\d+', str(x))
                return float(m.group(0)) if m else np.nan
            df["dem_political_lr"] = df[raw_pol].map(_first_int)

ensure_other_demogs()

# Make demogs categorical where appropriate
for cat_col in ["dem_age_band", "dem_gender", "dem_education", "dem_employment"]:
    if cat_col in df.columns and (df[cat_col].dtype == object or not str(df[cat_col].dtype).startswith("category")):
        df[cat_col] = df[cat_col].astype("category")

# -----------------------------
# Column sets (as created in your build step)
# -----------------------------
CDR_ENV = present([f"CDR attributions _{i}" for i in range(1,4)])
CDR_SOC = present([f"CDR attributions _{i}" for i in range(4,13)])
CDR_GOV = present([f"CDR attributions _{i}" for i in range(13,21)])
CDR_TOTAL = CDR_ENV + CDR_SOC + CDR_GOV

# Composite columns from build
DV_TOTAL = "CDR_total_mean"
DV_ENV   = "CDR_env_mean"
DV_SOC   = "CDR_soc_mean"
DV_GOV   = "CDR_gov_mean"

# ---------------------------------------------------------------------
# Predictor item blocks (use exact headers from the build step; with safe fallback)
# ---------------------------------------------------------------------
ENV_ITEMS   = [c for c in [f"CDR pred, envconsc_{i}" for i in range(1,5)] if c in df.columns]
SOC_ITEMS   = [c for c in [f"CDR pred, socconsc_{i}" for i in range(1,4)] if c in df.columns]
TRUST_ITEMS = [c for c in [f"CDR pred, trust_{i}"    for i in range(1,6)] if c in df.columns]
DIGIQ_ITEMS = [c for c in [f"CDR pred, digiQ_{i}"    for i in range(1,9)] if c in df.columns]

# Fallback (robust) if something's missing
if not (ENV_ITEMS and SOC_ITEMS and TRUST_ITEMS and DIGIQ_ITEMS):
    import unicodedata
    def _norm(s: str) -> str:
        if s is None: return ""
        s = unicodedata.normalize("NFKD", str(s))
        s = "".join(ch for ch in s if not unicodedata.combining(ch)).lower()
        return re.sub(r"[^a-z0-9]+", "", s)
    ALL = list(df.columns)
    def _grab(prefix, k):
        out=[]
        for i in range(1, k+1):
            nd = _norm(f"{prefix}_{i}")
            for col in ALL:
                if _norm(col).endswith(nd):
                    out.append(col); break
        return out
    ENV_ITEMS   = ENV_ITEMS   or _grab("cdrpredenvconsc", 4)
    SOC_ITEMS   = SOC_ITEMS   or _grab("cdrpredsocconsc", 3)
    TRUST_ITEMS = TRUST_ITEMS or _grab("cdrpredtrust",    5)
    DIGIQ_ITEMS = DIGIQ_ITEMS or _grab("cdrpreddigiq",    8)

# ---------- Scoring + composites ----------
def _coerce_likert(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series, errors="coerce")

def _normalize_boolish(v):
    if pd.isna(v): return np.nan
    t = str(v).strip().lower().replace(".", "")
    if "weet" in t or "begrijp" in t:
        return np.nan
    if t in {"juist","waar","true","1","yes","y","j","t"}:
        return True
    if t in {"onjuist","fout","false","0","nee","n","f"}:
        return False
    return np.nan

def score_digiq(raw: pd.DataFrame) -> pd.DataFrame:
    if raw.empty: return raw
    normed = raw.apply(lambda s: s.map(_normalize_boolish))
    out = pd.DataFrame(index=normed.index)
    for c in normed.columns:
        correct_true = not c.endswith("_1")  # digiQ_1 is False=correct; others True=correct
        out[c] = np.where(normed[c].isna(), np.nan,
                          (normed[c] == (True if correct_true else False)).astype(float))
    ordered = [f"CDR pred, digiQ_{i}" for i in range(1,9) if f"CDR pred, digiQ_{i}" in out.columns]
    return out[ordered] if ordered else out

env_df   = pd.DataFrame({c: _coerce_likert(df[c]) for c in ENV_ITEMS}) if ENV_ITEMS else pd.DataFrame(index=df.index)
soc_df   = pd.DataFrame({c: _coerce_likert(df[c]) for c in SOC_ITEMS}) if SOC_ITEMS else pd.DataFrame(index=df.index)
trust_df = pd.DataFrame({c: _coerce_likert(df[c]) for c in TRUST_ITEMS}) if TRUST_ITEMS else pd.DataFrame(index=df.index)
digiq_raw = pd.DataFrame({c: df[c] for c in DIGIQ_ITEMS}) if DIGIQ_ITEMS else pd.DataFrame(index=df.index)
digiq_df  = score_digiq(digiq_raw) if DIGIQ_ITEMS else pd.DataFrame(index=df.index)

def mean_score_row(row, min_frac=0.5):
    vals = row.values.astype(float)
    n = len(vals)
    ok = np.isfinite(vals)
    if ok.sum() >= max(1, int(np.ceil(n*min_frac))):
        return np.nanmean(vals[ok])
    return np.nan

# Create composites if missing
if "env_consc" not in df.columns and env_df.shape[1]:
    df["env_consc"] = env_df.apply(mean_score_row, axis=1)
if "soc_consc" not in df.columns and soc_df.shape[1]:
    df["soc_consc"] = soc_df.apply(mean_score_row, axis=1)
if "trust_mean" not in df.columns and trust_df.shape[1]:
    df["trust_mean"] = trust_df.apply(mean_score_row, axis=1)
if ("digiQ_total" not in df.columns or "digiQ_prop" not in df.columns) and digiq_df.shape[1]:
    df["digiQ_total"] = digiq_df.sum(axis=1, min_count=1)
    df["digiQ_prop"]  = df["digiQ_total"] / digiq_df.shape[1]

# -----------------------------
# Predictors list (now that we may have created them)
# -----------------------------
PREDICTORS = present(["env_consc", "soc_consc", "trust_mean", "digiQ_prop", "digiQ_total"])
DEMOGS = present(["dem_age_band", "dem_gender", "dem_education", "dem_employment", "dem_political_lr"])

# RQ1 blocks
RQ1_ENV = present([f"responsibility attri_{i}" for i in range(1,8+1)])
RQ1_SOC = present([f"resp_attribut_societ_{i}" for i in range(1,8+1)])
RQ1_GOV = present([f"resp_attribut_regula_{i}" for i in range(1,8+1)])

# -----------------------------
# 1) Sample overview (N, demogs)
# -----------------------------
N = len(df)

demo_blocks = []
if "dem_age_band" in df:    demo_blocks.append(("Leeftijd", vc_pct(df["dem_age_band"])))
if "dem_gender" in df:      demo_blocks.append(("Gender",   vc_pct(df["dem_gender"])))
if "dem_education" in df:   demo_blocks.append(("Opleiding",vc_pct(df["dem_education"])))
if "dem_employment" in df:  demo_blocks.append(("Werkstatus",vc_pct(df["dem_employment"])))
if "dem_political_lr" in df:
    demo_blocks.append(("Politiek L–R", df["dem_political_lr"].describe()[["count","mean","std","min","25%","50%","75%","max"]].to_frame().T.round(2)))

sample_lines = []
sample_lines.append("# 📊 Sample overview\n")
sample_lines.append(md_table(pd.DataFrame([{"Metric":"Analytic sample (kept)", "N": N}])))
sample_lines.append("\n## 👥 Demographics")

if demo_blocks:
    for title, tbl in demo_blocks:
        sample_lines.append(f"\n### {title}\n")
        if isinstance(tbl, pd.DataFrame):
            sample_lines.append(md_table(tbl.reset_index().rename(columns={"index": title})))
        else:
            sample_lines.append(str(tbl))
else:
    sample_lines.append("\n_No demographics found in dataset._")
    sample_lines.append("\n<details><summary>Debug: first headers</summary>\n\n```\n" + "\n".join(_repr_cols(40)) + "\n```\n</details>")

sample_md = "\n".join(sample_lines)
save_markdown("01_sample_overview.md", sample_md)

# -----------------------------
# 2) Reliability (α) — CDR outcomes (ESG)
# -----------------------------
alpha_rows = []
for label, cols in [("Environmental", CDR_ENV), ("Social", CDR_SOC), ("Governance", CDR_GOV)]:
    a, n = cronbach_alpha(df, cols)
    alpha_rows.append({"Domein": label, "k": len(cols), "N_alpha": n, "Cronbach_α": (None if pd.isna(a) else round(a,3))})
alpha_df = pd.DataFrame(alpha_rows)
alpha_md = "# 5️⃣ Betrouwbaarheid — Cronbach’s α (CDR outcomes)\n\n" + md_table(alpha_df)
save_markdown("02_reliability.md", alpha_md)

# -----------------------------
# 2b) Reliability (α) — Predictor blocks + diagnostics
# -----------------------------
def _item_total_corr(items_df: pd.DataFrame) -> pd.Series:
    x = items_df.dropna(axis=0, how="any")
    out = {}
    for c in x.columns:
        tot_minus = x.drop(columns=[c]).sum(axis=1)
        out[c] = np.corrcoef(x[c], tot_minus)[0,1] if x[c].nunique() > 1 else np.nan
    return pd.Series(out)

def _alpha_if_dropped(items_df: pd.DataFrame) -> pd.Series:
    x = items_df.dropna(axis=0, how="any")
    out = {}
    for c in x.columns:
        kept = x.drop(columns=[c])
        out[c] = cronbach_alpha(kept, kept.columns)[0]
    return pd.Series(out)

def reliability_table(name: str, items_df: pd.DataFrame) -> pd.DataFrame:
    if items_df.shape[1] < 2:
        return pd.DataFrame({"Scale":[name], "Item":["—"], "Item-total_r":[np.nan],
                             "Alpha_if_dropped":[np.nan], "k":[items_df.shape[1]],
                             "N_listwise":[int(items_df.dropna().shape[0])],
                             "Cronbach_alpha":[np.nan]})
    alpha, n = cronbach_alpha(items_df, items_df.columns)
    itc = _item_total_corr(items_df)
    aid = _alpha_if_dropped(items_df)
    out = (pd.DataFrame({
        "Item": items_df.columns,
        "Item-total_r": itc.reindex(items_df.columns).values,
        "Alpha_if_dropped": aid.reindex(items_df.columns).values,
    })
    .assign(Scale=name, k=items_df.shape[1], N_listwise=n, Cronbach_alpha=alpha))
    return out[["Scale","k","N_listwise","Cronbach_alpha","Item","Item-total_r","Alpha_if_dropped"]]

pred_tabs = []
if env_df.shape[1]:   pred_tabs.append(reliability_table("Environmental consciousness", env_df))
if soc_df.shape[1]:   pred_tabs.append(reliability_table("Social consciousness",        soc_df))
if trust_df.shape[1]: pred_tabs.append(reliability_table("Corporate trust",             trust_df))
if digiq_df.shape[1]: pred_tabs.append(reliability_table("Digital literacy (DigiQ, 0/1)", digiq_df))

if pred_tabs:
    pred_rel = pd.concat(pred_tabs, ignore_index=True)
    rd_write_csv_df(pred_rel, rd_join(OUT_DIR, "02b_reliability_predictors.csv"), index=False)
    blocks = []
    for scale in pred_rel["Scale"].drop_duplicates():
        sub = pred_rel[pred_rel["Scale"] == scale].copy()
        head = sub.iloc[0]
        blocks.append(f"## {scale}\n\n- Items (k): **{int(head['k'])}**  \n- N (listwise): **{int(head['N_listwise'])}**  \n- Cronbach’s α: **{head['Cronbach_alpha']:.3f}**\n")
        blocks.append(md_table(sub[["Item","Item-total_r","Alpha_if_dropped"]].round(3)))
        blocks.append("")
    save_markdown("02b_reliability_predictors.md", "# 🔧 Reliability — Predictors (Cronbach’s α)\n\n" + "\n".join(blocks))
else:
    save_markdown("02b_reliability_predictors.md", "# 🔧 Reliability — Predictors (Cronbach’s α)\n\n_No predictor items found._")

# -----------------------------
# 3) Descriptives
# -----------------------------
desc_cols = present([DV_TOTAL, DV_ENV, DV_SOC, DV_GOV] + PREDICTORS + ["CDR_total_mean"])
if desc_cols:
    desc = df[desc_cols].apply(pd.to_numeric, errors="coerce").describe().T
    desc = desc.rename(columns={"mean":"Mean","std":"SD","min":"Min","25%":"Q25","50%":"Median","75%":"Q75","max":"Max"})
    desc_md = "# 📈 Descriptives\n\n" + md_table(desc[["count","Mean","SD","Min","Q25","Median","Q75","Max"]].round(3).reset_index().rename(columns={"index":"Variable"}))
else:
    desc_md = "# 📈 Descriptives\n\n_No eligible numeric variables found._"
save_markdown("03_descriptives.md", desc_md)

# -----------------------------
# 4) Correlations
# -----------------------------
corr_cols = present([DV_TOTAL, DV_ENV, DV_SOC, DV_GOV, "env_consc", "soc_consc", "trust_mean", "digiQ_prop", "dem_political_lr"])
if len(corr_cols) >= 2:
    corr_df = df[corr_cols].apply(pd.to_numeric, errors="coerce").corr(min_periods=50)
    rd_write_csv_df(corr_df.round(3), rd_join(OUT_DIR, "04_correlations.csv"), index=True)
    print(f"[written] {rd_join(OUT_DIR, '04_correlations.csv')}")
    corr_preview = corr_df.round(3).reset_index().rename(columns={"index":"Var"})
    corr_md = "# 🔗 Correlations (Pearson)\n\n" + md_table(corr_preview.iloc[: min(15, len(corr_preview)), : min(8, len(corr_preview.columns))])
else:
    corr_md = "# 🔗 Correlations (Pearson)\n\n_Not enough variables for a correlation matrix._"
save_markdown("04_correlations.md", corr_md)

# -----------------------------
# 5) Regressions (OLS, HC3 SE)
# -----------------------------
def build_formula(dv):
    base = ["env_consc","soc_consc","trust_mean","digiQ_prop","dem_political_lr"]
    X = [c for c in base if c in df.columns and df[c].notna().sum() > 1]
    cats_raw = [c for c in ["dem_age_band","dem_gender","dem_education","dem_employment"]
                if c in df.columns and df[c].dropna().nunique() > 1]
    cats = [f"C({c})" for c in cats_raw]
    nums = [c for c in ["dem_political_lr"] if c in df.columns and df[c].notna().sum() > 1]
    terms = X + cats + [n for n in nums if n not in X]
    if not terms:
        return None
    return f"{dv} ~ " + " + ".join(terms)

def tidy_ols(model, model_name):
    params = model.params
    bse    = model.bse
    stat   = model.tvalues if hasattr(model, "tvalues") else (model.zvalues if hasattr(model, "zvalues") else None)
    ci     = model.conf_int()
    out = pd.DataFrame({
        "term": params.index,
        "coef": params.values,
        "se": bse.values,
        "stat": (stat.values if stat is not None else np.full_like(params.values, np.nan, dtype=float)),
        "p": model.pvalues.values,
        "ci_low": ci[0].values,
        "ci_high": ci[1].values,
    })
    out["model"] = model_name
    return out

def run_model(dv, name):
    formula = build_formula(dv)
    if not formula:
        return None, None
    try:
        model = smf.ols(formula, data=df).fit(cov_type="HC3")
    except Exception as e:
        note = f"# ⚠️ Regression skipped: {name}\n\n**Formula:** `{formula}`\n\n```\n{e}\n```"
        save_markdown(f"05_regressions_{name.replace(' ','_')}_ERROR.md", note)
        return None, None
    table = tidy_ols(model, name)
    return model, table

reg_tables = []
models_run = []
for dv, name in [(DV_TOTAL,"CDR Total"), (DV_ENV,"CDR Env"), (DV_SOC,"CDR Soc"), (DV_GOV,"CDR Gov")]:
    if dv in df.columns:
        m, t = run_model(dv, name)
        if t is not None:
            reg_tables.append(t)
            models_run.append((name, m))

def _stars(p):
    return "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""

if reg_tables:
    reg_all = pd.concat(reg_tables, ignore_index=True)
    reg_all_out = reg_all.copy()
    reg_all_out["sig"] = reg_all_out["p"].apply(_stars)
    reg_all_out[["coef","se","stat","p","ci_low","ci_high"]] = reg_all_out[["coef","se","stat","p","ci_low","ci_high"]].round(4)
    rd_write_csv_df(reg_all_out, rd_join(OUT_DIR, "05_regressions.csv"), index=False)
    reg_md = "# 📉 OLS regressions (HC3 SE)\n\n" + md_table(
        reg_all_out[["model","term","coef","se","stat","p","sig","ci_low","ci_high"]]
        .rename(columns={"coef":"Coef","se":"SE","stat":"t/z","p":"p","ci_low":"CI_low","ci_high":"CI_high"})
    )
    save_markdown("05_regressions.md", reg_md)
else:
    save_markdown("05_regressions.md", "# 📉 OLS regressions\n\n_No eligible models (missing predictors/DVs)._")

# -----------------------------
# 6) Responsibility attribution (RQ1)
# -----------------------------
def summarize_block(cols, title):
    if not cols: return f"## {title}\n\n_Geen kolommen gevonden._"
    sub = df[cols].apply(pd.to_numeric, errors="coerce")
    means = sub.mean(axis=0).sort_values(ascending=False).round(3)
    tbl = pd.DataFrame({"actor_item": means.index, "mean": means.values})
    tbl["rank"] = np.arange(1, len(tbl)+1)
    return f"## {title}\n\n" + md_table(tbl)

rq1_parts = []
rq1_parts.append(summarize_block(RQ1_ENV, "RQ1 — Environment: perceived responsibility (higher = more responsibility)"))
rq1_parts.append(summarize_block(RQ1_SOC, "RQ1 — Social: perceived responsibility"))
rq1_parts.append(summarize_block(RQ1_GOV, "RQ1 — Governance/Ethical use: perceived responsibility"))

rq1_md = "# 🧭 Responsibility Attributions\n\n" + "\n\n".join(rq1_parts)
save_markdown("06_responsibility_attribution.md", rq1_md)

# -----------------------------
# 7) Mini preregistration scaffold
# -----------------------------
alpha_md_inline = md_table(pd.DataFrame([
    {"Domein":"Environmental","k":len(CDR_ENV),"N_alpha":""},
    {"Domein":"Social","k":len(CDR_SOC),"N_alpha":""},
    {"Domein":"Governance","k":len(CDR_GOV),"N_alpha":""},
]).assign(**{"Cronbach_α":""}))

prereg = f"""
# 📝 Preregistration scaffold (draft)

## Outcomes
- **CDR_total_mean** (primary), **CDR_env_mean**, **CDR_soc_mean**, **CDR_gov_mean** (secondary).

## Predictors & Covariates
- Predictors: env_consc, soc_consc, trust_mean, digiQ_prop.
- Covariates: dem_age_band, dem_gender, dem_education, dem_employment, dem_political_lr (as available).

## Reliability
{alpha_md_inline}

- See also **02b_reliability_predictors.md** for predictor reliability and diagnostics.

## Models
- **M1 (Primary):** CDR_total_mean ~ z(env_consc) + z(soc_consc) + z(trust_mean) + z(digiQ_prop) + demographics.
- **M2–M4:** Same predictors with CDR_env_mean, CDR_soc_mean, CDR_gov_mean.

(Where z(x) denotes standardization; in the quick results above we reported unstandardized β with HC3 SE.)

## Missing data
- Listwise deletion within each model. No imputation for prereg primary; sensitivity analyses optional.

## Responsibility attribution (RQ1)
- Report actor-level means within each domain; interpret rank order patterns.
"""
save_markdown("07_prereg_scaffold.md", prereg)

# -----------------------------
# 8) Stitch single report (read REMOTE parts to inline full content)
# -----------------------------
def make_toc(md: str) -> str:
    lines = []
    for line in md.splitlines():
        m = re.match(r"^(#{1,3})\s+(.*)", line.strip())
        if m:
            level = len(m.group(1))
            title = m.group(2).strip()
            anchor = re.sub(r"[^\w\- ]", "", title).strip().lower().replace(" ", "-")
            lines.append(f"{'  '*(level-1)}- [{title}](#{anchor})")
    return "\n".join(lines)

SECTIONS = [
    "01_sample_overview.md",
    "02_reliability.md",
    "02b_reliability_predictors.md",
    "03_descriptives.md",
    "04_correlations.md",
    "05_regressions.md",
    "06_responsibility_attribution.md",
    "07_prereg_scaffold.md",
    # Include codebooks from the build step if present
    "codebook_predictors.md",
    "codebook_digiq_key.md",
]

def _read_remote_or_placeholder(fname: str) -> str:
    try:
        return rd_read_text(rd_join(OUT_DIR, fname), encoding="utf-8")
    except Exception:
        return f"# {fname}\n\n_Missing on remote: {fname}_"

parts_text = []
for fname in SECTIONS:
    txt = _read_remote_or_placeholder(fname)
    if not txt.startswith("\n"):
        txt = "\n" + txt
    parts_text.append(f"<!-- ⤷ {fname} -->\n{txt}")

body = "\n".join(parts_text).strip()

report = f"""# CDR — Analysis Report (prereg order)

_This document stitches the individual outputs into a single narrative, matching our preregistration structure._

## Table of Contents
{make_toc(body)}

---

{body}
"""

save_markdown("00_REPORT.md", report)
print("[written] stitched report with inline section contents -> 00_REPORT.md")

print("\nDone. Open the remote 'analysis_out' folder for Markdown + CSV outputs.\n")


[written] ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/01_sample_overview.md
[written] ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/02_reliability.md
[written] ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/02b_reliability_predictors.md
[written] ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/03_descriptives.md
[written] ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/04_correlations.csv
[written] ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/04_correlations.md
[written] ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/05_regressions.md
[written] ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/06_responsibility_attribution.md
[written] ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/07_prereg_scaffold.md
[written] ASCOR-FMG-14116-CDR (Projectfolder)/analysis_out/00_REPORT.md
[written] stitched report with inline section contents -> 00_REPORT.md

Done. Open the remote 'analysis_out' folder for Markdown + CSV outputs.



In [49]:
# Viewer — stitched report + individual sections + data previews
from IPython.display import Markdown, display
import pandas as pd

from rd_io import OUT_DIR, rd_join, rd_read_text, rd_read_csv_df

def try_read_md(remote_path: str) -> str | None:
    try:
        return rd_read_text(remote_path, encoding="utf-8")
    except Exception:
        return None

# 1) Full stitched report
report_path = rd_join(OUT_DIR, "00_REPORT.md")
report_text = try_read_md(report_path)

if report_text is not None:
    display(Markdown(
        "# 📓 Notebook View — Full Report\n\n"
        "_This is the stitched version (same as `00_REPORT.md`), rendered inline for quick reading._"
    ))
    display(Markdown(report_text))
else:
    display(Markdown("**Note:** Remote `00_REPORT.md` not found. I’ll render the individual sections below."))

# 2) Individual sections (includes predictor reliability + codebooks)
ORDER = [
    "01_sample_overview.md",
    "02_reliability.md",
    "02b_reliability_predictors.md",
    "03_descriptives.md",
    "04_correlations.md",
    "05_regressions.md",
    "06_responsibility_attribution.md",
    # "07_prereg_scaffold.md",
    "codebook_predictors.md",
    "codebook_digiq_key.md",
]

display(Markdown("---"))
display(Markdown("# 🔎 Individual Sections"))
for fname in ORDER:
    p = rd_join(OUT_DIR, fname)
    text = try_read_md(p)
    if text is not None:
        display(Markdown(f"---\n\n### ⤷ {fname}\n"))
        display(Markdown(text))
    else:
        display(Markdown(f"> _Missing (remote): {fname}_"))

# 3) (Optional) Data previews
# 3a) Correlations matrix preview
try:
    corr_remote = rd_join(OUT_DIR, "04_correlations.csv")
    df_corr = rd_read_csv_df(corr_remote, index_col=0)
    display(Markdown("### 📊 Correlations (DataFrame view)"))
    import caas_jupyter_tools
    caas_jupyter_tools.display_dataframe_to_user("Correlations (04_correlations.csv)", df_corr)
except Exception:
    pass

# 3b) Predictor reliability diagnostics (item-level) preview
try:
    pred_rel_csv = rd_join(OUT_DIR, "02b_reliability_predictors.csv")
    df_pred_rel = rd_read_csv_df(pred_rel_csv)
    display(Markdown("### 🔧 Predictor Reliability — Diagnostics (CSV view)"))
    import caas_jupyter_tools
    caas_jupyter_tools.display_dataframe_to_user("Predictor reliability (02b_reliability_predictors.csv)", df_pred_rel)
except Exception:
    pass


# 📓 Notebook View — Full Report

_This is the stitched version (same as `00_REPORT.md`), rendered inline for quick reading._

# CDR — Analysis Report (prereg order)

_This document stitches the individual outputs into a single narrative, matching our preregistration structure._

## Table of Contents
- [📊 Sample overview](#sample-overview)
  - [👥 Demographics](#demographics)
    - [Leeftijd](#leeftijd)
    - [Gender](#gender)
    - [Opleiding](#opleiding)
    - [Werkstatus](#werkstatus)
    - [Politiek L–R](#politiek-lr)
- [5️⃣ Betrouwbaarheid — Cronbach’s α (CDR outcomes)](#5-betrouwbaarheid--cronbachs-α-cdr-outcomes)
- [🔧 Reliability — Predictors (Cronbach’s α)](#reliability--predictors-cronbachs-α)
  - [Environmental consciousness](#environmental-consciousness)
  - [Social consciousness](#social-consciousness)
  - [Corporate trust](#corporate-trust)
  - [Digital literacy (DigiQ, 0/1)](#digital-literacy-digiq-01)
- [📈 Descriptives](#descriptives)
- [🔗 Correlations (Pearson)](#correlations-pearson)
- [📉 OLS regressions (HC3 SE)](#ols-regressions-hc3-se)
- [🧭 Responsibility Attributions](#responsibility-attributions)
  - [RQ1 — Environment: perceived responsibility (higher = more responsibility)](#rq1--environment-perceived-responsibility-higher--more-responsibility)
  - [RQ1 — Social: perceived responsibility](#rq1--social-perceived-responsibility)
  - [RQ1 — Governance/Ethical use: perceived responsibility](#rq1--governanceethical-use-perceived-responsibility)
- [📝 Preregistration scaffold (draft)](#preregistration-scaffold-draft)
  - [Outcomes](#outcomes)
  - [Predictors & Covariates](#predictors--covariates)
  - [Reliability](#reliability)
  - [Models](#models)
  - [Missing data](#missing-data)
  - [Responsibility attribution (RQ1)](#responsibility-attribution-rq1)
- [Codebook: Predictors](#codebook-predictors)
- [Codebook: DigiQ](#codebook-digiq)

---

<!-- ⤷ 01_sample_overview.md -->

# 📊 Sample overview

| Metric | N |
|:--|--:|
| Analytic sample (kept) | 2 196 |

## 👥 Demographics

### Leeftijd

| dem_age_band | n | % |
|:--|--:|--:|
| 18–24 | 234 | 10.7 |
| 25–34 | 488 | 22.2 |
| 35–44 | 440 | 20 |
| 45–54 | 510 | 23.2 |
| 55–65 | 524 | 23.9 |

### Gender

| dem_gender | n | % |
|:--|--:|--:|
| Man | 1 075 | 49 |
| Non-binair | 7 | 0.3 |
| Vrouw | 1 110 | 50.5 |
| Zeg ik liever niet | 4 | 0.2 |

### Opleiding

| dem_education | n | % |
|:--|--:|--:|
| Basisschool, lager onderwijs | 39 | 1.8 |
| Gepromoveerd (dr.) | 24 | 1.1 |
| Hoger algemeen en voorbereidend wetenschappelijk onderwijs (HAVO, VWO, HBS, Gymnasium, Atheneum) | 158 | 7.2 |
| Hoger beroepsonderwijs (HBO, HTS, HEAO, Kweekschool, Sociale of Pedagogische Academie) | 625 | 28.5 |
| Lager beroepsonderwijs (VMBO-B, VMBO-K, praktijkonderwijs, VGLO, LAVO, LTS, Huishoudschool) | 172 | 7.8 |
| Middelbaar algemeen beroepsonderwijs (MBO, MTS) | 374 | 17 |
| Middelbaar algemeen voortgezet onderwijs (VMBO-T, MAVO, MULO, 3-jarige HBS) | 231 | 10.5 |
| Universitaire Bachelor (BA, BSc) | 260 | 11.8 |
| Universitaire Master (drs., MA, MSc, mr., ir) | 313 | 14.3 |

### Werkstatus

| dem_employment | n | % |
|:--|--:|--:|
| Anders, namelijk: | 58 | 2.6 |
| Ik ben arbeidsongeschikt | 187 | 8.5 |
| Ik ben met (vervroegd) pensioen | 46 | 2.1 |
| Ik ben student/scholier   | 95 | 4.3 |
| Ik ben uitzendkracht / oproepkracht / invalkracht | 22 | 1 |
| Ik ben werkloos | 112 | 5.1 |
| Ik ben zelfstandig ondernemer / zzp’er | 193 | 8.8 |
| Ik werk in loondienst (vast of tijdelijk contract) | 1 483 | 67.5 |

### Politiek L–R

| Politiek L–R | count | mean | std | min | 25% | 50% | 75% | max |
|:--|--:|--:|--:|--:|--:|--:|--:|--:|
| dem_political_lr | 2 191 | 5.78 | 2.21 | 1 | 4 | 6 | 7 | 10 |
<!-- ⤷ 02_reliability.md -->

# 5️⃣ Betrouwbaarheid — Cronbach’s α (CDR outcomes)

| Domein | k | N_alpha | Cronbach_α |
|:--|--:|--:|--:|
| Environmental | 3 | 2 196 | 0.831 |
| Social | 9 | 2 196 | 0.913 |
| Governance | 8 | 2 196 | 0.943 |
<!-- ⤷ 02b_reliability_predictors.md -->

# 🔧 Reliability — Predictors (Cronbach’s α)

## Environmental consciousness

- Items (k): **4**  
- N (listwise): **2196**  
- Cronbach’s α: **0.847**

| Item | Item-total_r | Alpha_if_dropped |
|:--|--:|--:|
| CDR pred, envconsc_1 | 0.588 | 0.852 |
| CDR pred, envconsc_2 | 0.71 | 0.795 |
| CDR pred, envconsc_3 | 0.732 | 0.785 |
| CDR pred, envconsc_4 | 0.721 | 0.79 |

## Social consciousness

- Items (k): **3**  
- N (listwise): **2196**  
- Cronbach’s α: **0.788**

| Item | Item-total_r | Alpha_if_dropped |
|:--|--:|--:|
| CDR pred, socconsc_1 | 0.598 | 0.745 |
| CDR pred, socconsc_2 | 0.653 | 0.686 |
| CDR pred, socconsc_3 | 0.638 | 0.702 |

## Corporate trust

- Items (k): **5**  
- N (listwise): **2196**  
- Cronbach’s α: **0.925**

| Item | Item-total_r | Alpha_if_dropped |
|:--|--:|--:|
| CDR pred, trust_1 | 0.834 | 0.902 |
| CDR pred, trust_2 | 0.819 | 0.905 |
| CDR pred, trust_3 | 0.823 | 0.904 |
| CDR pred, trust_4 | 0.735 | 0.921 |
| CDR pred, trust_5 | 0.812 | 0.907 |

## Digital literacy (DigiQ, 0/1)

- Items (k): **8**  
- N (listwise): **714**  
- Cronbach’s α: **0.489**

| Item | Item-total_r | Alpha_if_dropped |
|:--|--:|--:|
| CDR pred, digiQ_1 | 0.07 | 0.547 |
| CDR pred, digiQ_2 | 0.18 | 0.472 |
| CDR pred, digiQ_3 | 0.408 | 0.426 |
| CDR pred, digiQ_4 | 0.199 | 0.465 |
| CDR pred, digiQ_5 | 0.346 | 0.423 |
| CDR pred, digiQ_6 | 0.21 | 0.463 |
| CDR pred, digiQ_7 | 0.318 | 0.418 |
| CDR pred, digiQ_8 | 0.309 | 0.434 |

<!-- ⤷ 03_descriptives.md -->

# 📈 Descriptives

| Variable | count | Mean | SD | Min | Q25 | Median | Q75 | Max |
|:--|--:|--:|--:|--:|--:|--:|--:|--:|
| CDR_total_mean | 2 196 | 5.461 | 1.083 | 1 | 4.55 | 5.6 | 6.362 | 7 |
| CDR_env_mean | 2 196 | 5.333 | 1.239 | 1 | 4.333 | 5.333 | 6.333 | 7 |
| CDR_soc_mean | 2 196 | 5.362 | 1.093 | 1 | 4.444 | 5.444 | 6.222 | 7 |
| CDR_gov_mean | 2 196 | 5.621 | 1.165 | 1 | 4.625 | 5.875 | 6.75 | 7 |
| env_consc | 2 196 | 5.117 | 1.196 | 1 | 4.25 | 5.25 | 6 | 7 |
| soc_consc | 2 196 | 4.991 | 1.116 | 1 | 4.333 | 5 | 5.667 | 7 |
| trust_mean | 2 196 | 3.828 | 1.333 | 1 | 3 | 4 | 4.6 | 7 |
| digiQ_prop | 2 196 | 0.67 | 0.296 | 0 | 0.5 | 0.75 | 0.875 | 1 |
| digiQ_total | 2 196 | 5.321 | 2.383 | 0 | 4 | 6 | 7 | 8 |
| CDR_total_mean | 2 196 | 5.461 | 1.083 | 1 | 4.55 | 5.6 | 6.362 | 7 |
<!-- ⤷ 04_correlations.md -->

# 🔗 Correlations (Pearson)

| Var | CDR_total_mean | CDR_env_mean | CDR_soc_mean | CDR_gov_mean | env_consc | soc_consc | trust_mean |
|:--|--:|--:|--:|--:|--:|--:|--:|
| CDR_total_mean | 1 | 0.855 | 0.968 | 0.961 | 0.533 | 0.429 | -0.05 |
| CDR_env_mean | 0.855 | 1 | 0.786 | 0.76 | 0.588 | 0.424 | -0.033 |
| CDR_soc_mean | 0.968 | 0.786 | 1 | 0.882 | 0.508 | 0.423 | -0.005 |
| CDR_gov_mean | 0.961 | 0.76 | 0.882 | 1 | 0.47 | 0.382 | -0.099 |
| env_consc | 0.533 | 0.588 | 0.508 | 0.47 | 1 | 0.604 | 0.119 |
| soc_consc | 0.429 | 0.424 | 0.423 | 0.382 | 0.604 | 1 | 0.215 |
| trust_mean | -0.05 | -0.033 | -0.005 | -0.099 | 0.119 | 0.215 | 1 |
| digiQ_prop | 0.475 | 0.389 | 0.431 | 0.494 | 0.265 | 0.181 | -0.239 |
| dem_political_lr | -0.097 | -0.127 | -0.084 | -0.087 | -0.199 | -0.108 | 0.164 |
<!-- ⤷ 05_regressions.md -->

# 📉 OLS regressions (HC3 SE)

| model | term | Coef | SE | t/z | p | sig | CI_low | CI_high |
|:--|:--|--:|--:|--:|--:|:--|--:|--:|
| CDR Total | Intercept | 1.915 | 0.235 | 8.14 | 0 | *** | 1.454 | 2.376 |
| CDR Total | C(dem_age_band)[T.25–34] | 0.079 | 0.067 | 1.177 | 0.239 |  | -0.053 | 0.211 |
| CDR Total | C(dem_age_band)[T.35–44] | 0.169 | 0.07 | 2.406 | 0.016 | * | 0.031 | 0.306 |
| CDR Total | C(dem_age_band)[T.45–54] | 0.179 | 0.07 | 2.558 | 0.011 | * | 0.042 | 0.317 |
| CDR Total | C(dem_age_band)[T.55–65] | 0.445 | 0.071 | 6.253 | 0 | *** | 0.306 | 0.585 |
| CDR Total | C(dem_gender)[T.Non-binair] | 0.425 | 0.399 | 1.066 | 0.286 |  | -0.357 | 1.208 |
| CDR Total | C(dem_gender)[T.Vrouw] | 0.102 | 0.036 | 2.868 | 0.004 | ** | 0.032 | 0.172 |
| CDR Total | C(dem_gender)[T.Zeg ik liever niet] | 0.185 | 0.324 | 0.57 | 0.569 |  | -0.45 | 0.819 |
| CDR Total | C(dem_education)[T.Gepromoveerd (dr.)] | 0.239 | 0.183 | 1.303 | 0.193 |  | -0.12 | 0.598 |
| CDR Total | C(dem_education)[T.Hoger algemeen en voorbereidend wetenschappelijk onderwijs (HAVO, VWO, HBS, Gymnasium, Atheneum)] | 0.224 | 0.17 | 1.32 | 0.187 |  | -0.109 | 0.557 |
| CDR Total | C(dem_education)[T.Hoger beroepsonderwijs (HBO, HTS, HEAO, Kweekschool, Sociale of Pedagogische Academie)] | 0.221 | 0.158 | 1.403 | 0.161 |  | -0.088 | 0.53 |
| CDR Total | C(dem_education)[T.Lager beroepsonderwijs (VMBO-B, VMBO-K, praktijkonderwijs, VGLO, LAVO, LTS, Huishoudschool)] | 0.15 | 0.169 | 0.892 | 0.372 |  | -0.18 | 0.481 |
| CDR Total | C(dem_education)[T.Middelbaar algemeen beroepsonderwijs (MBO, MTS)] | 0.167 | 0.159 | 1.047 | 0.295 |  | -0.145 | 0.478 |
| CDR Total | C(dem_education)[T.Middelbaar algemeen voortgezet onderwijs (VMBO-T, MAVO, MULO, 3-jarige HBS)] | 0.096 | 0.164 | 0.589 | 0.556 |  | -0.224 | 0.417 |
| CDR Total | C(dem_education)[T.Universitaire Bachelor (BA, BSc)] | 0.21 | 0.16 | 1.317 | 0.188 |  | -0.103 | 0.524 |
| CDR Total | C(dem_education)[T.Universitaire Master (drs., MA, MSc, mr., ir)] | 0.099 | 0.162 | 0.611 | 0.541 |  | -0.218 | 0.415 |
| CDR Total | C(dem_employment)[T.Ik ben arbeidsongeschikt] | -0.094 | 0.13 | -0.726 | 0.468 |  | -0.349 | 0.16 |
| CDR Total | C(dem_employment)[T.Ik ben met (vervroegd) pensioen] | -0.188 | 0.173 | -1.089 | 0.276 |  | -0.527 | 0.151 |
| CDR Total | C(dem_employment)[T.Ik ben student/scholier  ] | -0.059 | 0.16 | -0.369 | 0.712 |  | -0.372 | 0.254 |
| CDR Total | C(dem_employment)[T.Ik ben uitzendkracht / oproepkracht / invalkracht] | 0.049 | 0.179 | 0.276 | 0.782 |  | -0.301 | 0.4 |
| CDR Total | C(dem_employment)[T.Ik ben werkloos] | 0.047 | 0.143 | 0.327 | 0.744 |  | -0.233 | 0.327 |
| CDR Total | C(dem_employment)[T.Ik ben zelfstandig ondernemer / zzp’er] | -0.182 | 0.128 | -1.425 | 0.154 |  | -0.432 | 0.068 |
| CDR Total | C(dem_employment)[T.Ik werk in loondienst (vast of tijdelijk contract)] | -0.15 | 0.115 | -1.312 | 0.19 |  | -0.375 | 0.074 |
| CDR Total | env_consc | 0.311 | 0.023 | 13.67 | 0 | *** | 0.266 | 0.355 |
| CDR Total | soc_consc | 0.163 | 0.022 | 7.363 | 0 | *** | 0.12 | 0.207 |
| CDR Total | trust_mean | -0.024 | 0.016 | -1.556 | 0.12 |  | -0.055 | 0.006 |
| CDR Total | digiQ_prop | 1.211 | 0.061 | 19.868 | 0 | *** | 1.092 | 1.331 |
| CDR Total | dem_political_lr | 0.022 | 0.009 | 2.59 | 0.01 | ** | 0.005 | 0.039 |
| CDR Env | Intercept | 1.406 | 0.276 | 5.088 | 0 | *** | 0.864 | 1.947 |
| CDR Env | C(dem_age_band)[T.25–34] | 0.062 | 0.086 | 0.717 | 0.474 |  | -0.107 | 0.23 |
| CDR Env | C(dem_age_band)[T.35–44] | 0.156 | 0.087 | 1.781 | 0.075 |  | -0.016 | 0.327 |
| CDR Env | C(dem_age_band)[T.45–54] | 0.182 | 0.088 | 2.075 | 0.038 | * | 0.01 | 0.354 |
| CDR Env | C(dem_age_band)[T.55–65] | 0.381 | 0.09 | 4.211 | 0 | *** | 0.204 | 0.558 |
| CDR Env | C(dem_gender)[T.Non-binair] | 0.271 | 0.348 | 0.779 | 0.436 |  | -0.411 | 0.953 |
| CDR Env | C(dem_gender)[T.Vrouw] | 0.062 | 0.041 | 1.528 | 0.127 |  | -0.018 | 0.141 |
| CDR Env | C(dem_gender)[T.Zeg ik liever niet] | 0.412 | 0.517 | 0.798 | 0.425 |  | -0.601 | 1.425 |
| CDR Env | C(dem_education)[T.Gepromoveerd (dr.)] | 0.293 | 0.2 | 1.464 | 0.143 |  | -0.099 | 0.686 |
| CDR Env | C(dem_education)[T.Hoger algemeen en voorbereidend wetenschappelijk onderwijs (HAVO, VWO, HBS, Gymnasium, Atheneum)] | 0.046 | 0.196 | 0.235 | 0.814 |  | -0.338 | 0.43 |
| CDR Env | C(dem_education)[T.Hoger beroepsonderwijs (HBO, HTS, HEAO, Kweekschool, Sociale of Pedagogische Academie)] | 0.051 | 0.18 | 0.282 | 0.778 |  | -0.302 | 0.404 |
| CDR Env | C(dem_education)[T.Lager beroepsonderwijs (VMBO-B, VMBO-K, praktijkonderwijs, VGLO, LAVO, LTS, Huishoudschool)] | 0.058 | 0.192 | 0.302 | 0.763 |  | -0.318 | 0.433 |
| CDR Env | C(dem_education)[T.Middelbaar algemeen beroepsonderwijs (MBO, MTS)] | 0.014 | 0.181 | 0.077 | 0.939 |  | -0.341 | 0.369 |
| CDR Env | C(dem_education)[T.Middelbaar algemeen voortgezet onderwijs (VMBO-T, MAVO, MULO, 3-jarige HBS)] | 0.037 | 0.189 | 0.194 | 0.847 |  | -0.334 | 0.407 |
| CDR Env | C(dem_education)[T.Universitaire Bachelor (BA, BSc)] | 0.147 | 0.183 | 0.801 | 0.423 |  | -0.212 | 0.505 |
| CDR Env | C(dem_education)[T.Universitaire Master (drs., MA, MSc, mr., ir)] | -0.035 | 0.182 | -0.189 | 0.85 |  | -0.392 | 0.323 |
| CDR Env | C(dem_employment)[T.Ik ben arbeidsongeschikt] | 0.117 | 0.153 | 0.77 | 0.441 |  | -0.182 | 0.417 |
| CDR Env | C(dem_employment)[T.Ik ben met (vervroegd) pensioen] | -0.088 | 0.221 | -0.401 | 0.689 |  | -0.521 | 0.344 |
| CDR Env | C(dem_employment)[T.Ik ben student/scholier  ] | 0.104 | 0.196 | 0.532 | 0.595 |  | -0.28 | 0.488 |
| CDR Env | C(dem_employment)[T.Ik ben uitzendkracht / oproepkracht / invalkracht] | 0.262 | 0.224 | 1.171 | 0.242 |  | -0.176 | 0.7 |
| CDR Env | C(dem_employment)[T.Ik ben werkloos] | 0.245 | 0.174 | 1.41 | 0.158 |  | -0.095 | 0.585 |
| CDR Env | C(dem_employment)[T.Ik ben zelfstandig ondernemer / zzp’er] | 0.018 | 0.154 | 0.12 | 0.905 |  | -0.283 | 0.32 |
| CDR Env | C(dem_employment)[T.Ik werk in loondienst (vast of tijdelijk contract)] | 0.07 | 0.14 | 0.498 | 0.619 |  | -0.204 | 0.344 |
| CDR Env | env_consc | 0.473 | 0.028 | 16.689 | 0 | *** | 0.417 | 0.528 |
| CDR Env | soc_consc | 0.128 | 0.028 | 4.548 | 0 | *** | 0.073 | 0.183 |
| CDR Env | trust_mean | -0.043 | 0.018 | -2.337 | 0.019 | * | -0.079 | -0.007 |
| CDR Env | digiQ_prop | 0.96 | 0.072 | 13.329 | 0 | *** | 0.819 | 1.101 |
| CDR Env | dem_political_lr | 0.01 | 0.01 | 1.024 | 0.306 |  | -0.009 | 0.029 |
| CDR Soc | Intercept | 1.866 | 0.242 | 7.707 | 0 | *** | 1.391 | 2.341 |
| CDR Soc | C(dem_age_band)[T.25–34] | 0.112 | 0.073 | 1.53 | 0.126 |  | -0.031 | 0.255 |
| CDR Soc | C(dem_age_band)[T.35–44] | 0.194 | 0.075 | 2.579 | 0.01 | ** | 0.046 | 0.341 |
| CDR Soc | C(dem_age_band)[T.45–54] | 0.202 | 0.074 | 2.717 | 0.007 | ** | 0.056 | 0.347 |
| CDR Soc | C(dem_age_band)[T.55–65] | 0.454 | 0.076 | 5.973 | 0 | *** | 0.305 | 0.604 |
| CDR Soc | C(dem_gender)[T.Non-binair] | 0.552 | 0.432 | 1.278 | 0.201 |  | -0.295 | 1.399 |
| CDR Soc | C(dem_gender)[T.Vrouw] | 0.116 | 0.038 | 3.077 | 0.002 | ** | 0.042 | 0.19 |
| CDR Soc | C(dem_gender)[T.Zeg ik liever niet] | 0.3 | 0.345 | 0.872 | 0.383 |  | -0.375 | 0.976 |
| CDR Soc | C(dem_education)[T.Gepromoveerd (dr.)] | 0.028 | 0.215 | 0.132 | 0.895 |  | -0.393 | 0.45 |
| CDR Soc | C(dem_education)[T.Hoger algemeen en voorbereidend wetenschappelijk onderwijs (HAVO, VWO, HBS, Gymnasium, Atheneum)] | 0.153 | 0.178 | 0.86 | 0.39 |  | -0.196 | 0.501 |
| CDR Soc | C(dem_education)[T.Hoger beroepsonderwijs (HBO, HTS, HEAO, Kweekschool, Sociale of Pedagogische Academie)] | 0.146 | 0.165 | 0.881 | 0.378 |  | -0.178 | 0.47 |
| CDR Soc | C(dem_education)[T.Lager beroepsonderwijs (VMBO-B, VMBO-K, praktijkonderwijs, VGLO, LAVO, LTS, Huishoudschool)] | 0.127 | 0.175 | 0.723 | 0.47 |  | -0.217 | 0.47 |
| CDR Soc | C(dem_education)[T.Middelbaar algemeen beroepsonderwijs (MBO, MTS)] | 0.138 | 0.166 | 0.83 | 0.407 |  | -0.188 | 0.463 |
| CDR Soc | C(dem_education)[T.Middelbaar algemeen voortgezet onderwijs (VMBO-T, MAVO, MULO, 3-jarige HBS)] | 0.051 | 0.171 | 0.298 | 0.765 |  | -0.284 | 0.386 |
| CDR Soc | C(dem_education)[T.Universitaire Bachelor (BA, BSc)] | 0.103 | 0.168 | 0.609 | 0.543 |  | -0.228 | 0.433 |
| CDR Soc | C(dem_education)[T.Universitaire Master (drs., MA, MSc, mr., ir)] | -0.017 | 0.17 | -0.101 | 0.919 |  | -0.35 | 0.315 |
| CDR Soc | C(dem_employment)[T.Ik ben arbeidsongeschikt] | -0.127 | 0.134 | -0.944 | 0.345 |  | -0.39 | 0.136 |
| CDR Soc | C(dem_employment)[T.Ik ben met (vervroegd) pensioen] | -0.29 | 0.179 | -1.617 | 0.106 |  | -0.641 | 0.061 |
| CDR Soc | C(dem_employment)[T.Ik ben student/scholier  ] | -0.09 | 0.165 | -0.545 | 0.585 |  | -0.414 | 0.234 |
| CDR Soc | C(dem_employment)[T.Ik ben uitzendkracht / oproepkracht / invalkracht] | 0.081 | 0.192 | 0.425 | 0.671 |  | -0.294 | 0.457 |
| CDR Soc | C(dem_employment)[T.Ik ben werkloos] | -0.008 | 0.147 | -0.056 | 0.955 |  | -0.296 | 0.279 |
| CDR Soc | C(dem_employment)[T.Ik ben zelfstandig ondernemer / zzp’er] | -0.211 | 0.133 | -1.59 | 0.112 |  | -0.47 | 0.049 |
| CDR Soc | C(dem_employment)[T.Ik werk in loondienst (vast of tijdelijk contract)] | -0.172 | 0.117 | -1.464 | 0.143 |  | -0.402 | 0.058 |
| CDR Soc | env_consc | 0.29 | 0.024 | 12.061 | 0 | *** | 0.243 | 0.338 |
| CDR Soc | soc_consc | 0.175 | 0.024 | 7.318 | 0 | *** | 0.128 | 0.222 |
| CDR Soc | trust_mean | 0.008 | 0.017 | 0.481 | 0.63 |  | -0.025 | 0.041 |
| CDR Soc | digiQ_prop | 1.128 | 0.064 | 17.709 | 0 | *** | 1.003 | 1.253 |
| CDR Soc | dem_political_lr | 0.021 | 0.009 | 2.375 | 0.018 | * | 0.004 | 0.039 |
| CDR Gov | Intercept | 2.162 | 0.259 | 8.359 | 0 | *** | 1.655 | 2.668 |
| CDR Gov | C(dem_age_band)[T.25–34] | 0.049 | 0.078 | 0.633 | 0.526 |  | -0.103 | 0.202 |
| CDR Gov | C(dem_age_band)[T.35–44] | 0.145 | 0.081 | 1.797 | 0.072 |  | -0.013 | 0.303 |
| CDR Gov | C(dem_age_band)[T.45–54] | 0.153 | 0.08 | 1.908 | 0.056 |  | -0.004 | 0.31 |
| CDR Gov | C(dem_age_band)[T.55–65] | 0.459 | 0.082 | 5.614 | 0 | *** | 0.299 | 0.619 |
| CDR Gov | C(dem_gender)[T.Non-binair] | 0.341 | 0.432 | 0.789 | 0.43 |  | -0.506 | 1.188 |
| CDR Gov | C(dem_gender)[T.Vrouw] | 0.102 | 0.04 | 2.577 | 0.01 | ** | 0.024 | 0.18 |
| CDR Gov | C(dem_gender)[T.Zeg ik liever niet] | -0.031 | 0.286 | -0.109 | 0.913 |  | -0.592 | 0.529 |
| CDR Gov | C(dem_education)[T.Gepromoveerd (dr.)] | 0.455 | 0.203 | 2.244 | 0.025 | * | 0.058 | 0.852 |
| CDR Gov | C(dem_education)[T.Hoger algemeen en voorbereidend wetenschappelijk onderwijs (HAVO, VWO, HBS, Gymnasium, Atheneum)] | 0.371 | 0.188 | 1.97 | 0.049 | * | 0.002 | 0.739 |
| CDR Gov | C(dem_education)[T.Hoger beroepsonderwijs (HBO, HTS, HEAO, Kweekschool, Sociale of Pedagogische Academie)] | 0.37 | 0.176 | 2.107 | 0.035 | * | 0.026 | 0.714 |
| CDR Gov | C(dem_education)[T.Lager beroepsonderwijs (VMBO-B, VMBO-K, praktijkonderwijs, VGLO, LAVO, LTS, Huishoudschool)] | 0.212 | 0.185 | 1.144 | 0.253 |  | -0.151 | 0.575 |
| CDR Gov | C(dem_education)[T.Middelbaar algemeen beroepsonderwijs (MBO, MTS)] | 0.256 | 0.177 | 1.45 | 0.147 |  | -0.09 | 0.602 |
| CDR Gov | C(dem_education)[T.Middelbaar algemeen voortgezet onderwijs (VMBO-T, MAVO, MULO, 3-jarige HBS)] | 0.17 | 0.18 | 0.941 | 0.347 |  | -0.184 | 0.524 |
| CDR Gov | C(dem_education)[T.Universitaire Bachelor (BA, BSc)] | 0.356 | 0.178 | 1.995 | 0.046 | * | 0.006 | 0.706 |
| CDR Gov | C(dem_education)[T.Universitaire Master (drs., MA, MSc, mr., ir)] | 0.279 | 0.18 | 1.546 | 0.122 |  | -0.075 | 0.632 |
| CDR Gov | C(dem_employment)[T.Ik ben arbeidsongeschikt] | -0.137 | 0.142 | -0.966 | 0.334 |  | -0.416 | 0.141 |
| CDR Gov | C(dem_employment)[T.Ik ben met (vervroegd) pensioen] | -0.112 | 0.187 | -0.597 | 0.55 |  | -0.478 | 0.255 |
| CDR Gov | C(dem_employment)[T.Ik ben student/scholier  ] | -0.085 | 0.178 | -0.476 | 0.634 |  | -0.435 | 0.265 |
| CDR Gov | C(dem_employment)[T.Ik ben uitzendkracht / oproepkracht / invalkracht] | -0.066 | 0.193 | -0.343 | 0.732 |  | -0.444 | 0.312 |
| CDR Gov | C(dem_employment)[T.Ik ben werkloos] | 0.034 | 0.157 | 0.218 | 0.827 |  | -0.274 | 0.342 |
| CDR Gov | C(dem_employment)[T.Ik ben zelfstandig ondernemer / zzp’er] | -0.225 | 0.14 | -1.602 | 0.109 |  | -0.5 | 0.05 |
| CDR Gov | C(dem_employment)[T.Ik werk in loondienst (vast of tijdelijk contract)] | -0.208 | 0.126 | -1.651 | 0.099 |  | -0.456 | 0.039 |
| CDR Gov | env_consc | 0.272 | 0.025 | 10.786 | 0 | *** | 0.223 | 0.322 |
| CDR Gov | soc_consc | 0.163 | 0.025 | 6.659 | 0 | *** | 0.115 | 0.211 |
| CDR Gov | trust_mean | -0.054 | 0.017 | -3.182 | 0.002 | ** | -0.087 | -0.021 |
| CDR Gov | digiQ_prop | 1.399 | 0.069 | 20.352 | 0 | *** | 1.264 | 1.533 |
| CDR Gov | dem_political_lr | 0.028 | 0.009 | 2.927 | 0.003 | ** | 0.009 | 0.046 |
<!-- ⤷ 06_responsibility_attribution.md -->

# 🧭 Responsibility Attributions

## RQ1 — Environment: perceived responsibility (higher = more responsibility)

| actor_item | mean | rank |
|:--|--:|--:|
| responsibility attri_1 | 5.174 | 1 |
| responsibility attri_5 | 5.071 | 2 |
| responsibility attri_4 | 5.036 | 3 |
| responsibility attri_2 | 4.881 | 4 |
| responsibility attri_7 | 4.365 | 5 |
| responsibility attri_8 | 4.343 | 6 |
| responsibility attri_6 | 4.316 | 7 |
| responsibility attri_3 | 4.114 | 8 |

## RQ1 — Social: perceived responsibility

| actor_item | mean | rank |
|:--|--:|--:|
| resp_attribut_societ_1 | 5.14 | 1 |
| resp_attribut_societ_5 | 5.126 | 2 |
| resp_attribut_societ_4 | 5.117 | 3 |
| resp_attribut_societ_2 | 4.93 | 4 |
| resp_attribut_societ_8 | 4.524 | 5 |
| resp_attribut_societ_7 | 4.473 | 6 |
| resp_attribut_societ_6 | 4.379 | 7 |
| resp_attribut_societ_3 | 4.311 | 8 |

## RQ1 — Governance/Ethical use: perceived responsibility

| actor_item | mean | rank |
|:--|--:|--:|
| resp_attribut_regula_4 | 5.31 | 1 |
| resp_attribut_regula_5 | 5.298 | 2 |
| resp_attribut_regula_1 | 5.028 | 3 |
| resp_attribut_regula_2 | 4.775 | 4 |
| resp_attribut_regula_7 | 4.486 | 5 |
| resp_attribut_regula_8 | 4.441 | 6 |
| resp_attribut_regula_3 | 4.168 | 7 |
| resp_attribut_regula_6 | 4.118 | 8 |
<!-- ⤷ 07_prereg_scaffold.md -->

# 📝 Preregistration scaffold (draft)

## Outcomes
- **CDR_total_mean** (primary), **CDR_env_mean**, **CDR_soc_mean**, **CDR_gov_mean** (secondary).

## Predictors & Covariates
- Predictors: env_consc, soc_consc, trust_mean, digiQ_prop.
- Covariates: dem_age_band, dem_gender, dem_education, dem_employment, dem_political_lr (as available).

## Reliability
| Domein | k | N_alpha | Cronbach_α |
|:--|--:|:--|:--|
| Environmental | 3 |  |  |
| Social | 9 |  |  |
| Governance | 8 |  |  |

- See also **02b_reliability_predictors.md** for predictor reliability and diagnostics.

## Models
- **M1 (Primary):** CDR_total_mean ~ z(env_consc) + z(soc_consc) + z(trust_mean) + z(digiQ_prop) + demographics.
- **M2–M4:** Same predictors with CDR_env_mean, CDR_soc_mean, CDR_gov_mean.

(Where z(x) denotes standardization; in the quick results above we reported unstandardized β with HC3 SE.)

## Missing data
- Listwise deletion within each model. No imputation for prereg primary; sensitivity analyses optional.

## Responsibility attribution (RQ1)
- Report actor-level means within each domain; interpret rank order patterns.

<!-- ⤷ codebook_predictors.md -->


# Codebook: Predictors
|Variabele|Volledige NL-vraag (QSF-verrijkt of CSV)|
|:---------|:----------------------------------------|
|CDR pred, envconsc_1|In welke mate ben je het eens of oneens met de volgende stellingen? - Als ik de keuze heb tussen twee gelijke producten, koop ik altijd het product dat minder schadelijk is voor andere mensen en het milieu.|
|CDR pred, envconsc_2|In welke mate ben je het eens of oneens met de volgende stellingen? - Het is belangrijk om voor het milieu te zorgen.|
|CDR pred, envconsc_3|In welke mate ben je het eens of oneens met de volgende stellingen? - Ik probeer milieuvriendelijke keuzes te maken.|
|CDR pred, envconsc_4|In welke mate ben je het eens of oneens met de volgende stellingen? - Iedereen zou zich moeten inspannen om onze natuurlijke hulpbronnen te behouden.|
|CDR pred, socconsc_1|In welke mate ben je het eens of oneens met de volgende stellingen? - Ik ben een sociaal bewust persoon.|
|CDR pred, socconsc_2|In welke mate ben je het eens of oneens met de volgende stellingen? - Ik ben betrokken bij het verbeteren van het welzijn van de samenleving.|
|CDR pred, socconsc_3|In welke mate ben je het eens of oneens met de volgende stellingen? - Ik streef hoge ethische normen na.|
|CDR pred, trust_1|In welke mate ben je het eens of oneens met de volgende stellingen over bedrijven? - Ik kan erop vertrouwen dat bedrijven mensen eerlijk en rechtvaardig behandelen.|
|CDR pred, trust_2|In welke mate ben je het eens of oneens met de volgende stellingen over bedrijven? - Ik kan erop vertrouwen dat bedrijven rekening houden met mensen zoals mezelf als ze een belangrijke beslissing nemen.|
|CDR pred, trust_3|In welke mate ben je het eens of oneens met de volgende stellingen over bedrijven? - Ik kan erop vertrouwen dat bedrijven hun beloften nakomen.|
|CDR pred, trust_4|In welke mate ben je het eens of oneens met de volgende stellingen over bedrijven? - Ik heb veel vertrouwen in de vaardigheden van bedrijven.|
|CDR pred, trust_5|In welke mate ben je het eens of oneens met de volgende stellingen over bedrijven? - Ik kan erop vertrouwen dat bedrijven competent zijn om te doen wat ze zeggen dat ze zullen doen.|
<!-- ⤷ codebook_digiq_key.md -->


# Codebook: DigiQ
|Variabele|Volledige NL-vraag (QSF-verrijkt of CSV)|Opmerking/Key|
|:---------|:----------------------------------------|:-------------|
|CDR pred, digiQ_1|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - Iedereen krijgt dezelfde informatie als ze online naar dezelfde dingen zoeken.|Fout/Onjuist = Correct|
|CDR pred, digiQ_2|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - Voordat je een foto waarop een vriend duidelijk te zien is digitaal deelt, moet je altijd toestemming vragen.|Juist = Correct|
|CDR pred, digiQ_3|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - Sommige mensen worden betaald om producten te gebruiken in de video's die ze maken.|Juist = Correct|
|CDR pred, digiQ_4|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - Om je apparaten veiliger te houden, moet je updates altijd meteen installeren.|Juist = Correct|
|CDR pred, digiQ_5|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - Wat jij online doet, gebruiken bedrijven om hun producten en diensten te adverteren.|Juist = Correct|
|CDR pred, digiQ_6|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - Je slaapt slechter als je een smartphone of computer gebruikt vlak voor je naar bed gaat.|Juist = Correct|
|CDR pred, digiQ_7|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - In telefoons zitten stoffen die door mijnwerkers uit mijnen gehaald worden.|Juist = Correct|
|CDR pred, digiQ_8|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - Je online gedrag bepaalt wat je te zien krijgt op websites en apps voor nieuws en entertainment.|Juist = Correct|

**Scoring:** Correct=1; Incorrect=0; 'Ik weet het niet'=0; 'Ik begrijp de vraag niet'=NaN.


---

# 🔎 Individual Sections

---

### ⤷ 01_sample_overview.md


...



# Codebook: Predictors
|Variabele|Volledige NL-vraag (QSF-verrijkt of CSV)|
|:---------|:----------------------------------------|
|CDR pred, envconsc_1|In welke mate ben je het eens of oneens met de volgende stellingen? - Als ik de keuze heb tussen twee gelijke producten, koop ik altijd het product dat minder schadelijk is voor andere mensen en het milieu.|
|CDR pred, envconsc_2|In welke mate ben je het eens of oneens met de volgende stellingen? - Het is belangrijk om voor het milieu te zorgen.|
|CDR pred, envconsc_3|In welke mate ben je het eens of oneens met de volgende stellingen? - Ik probeer milieuvriendelijke keuzes te maken.|
|CDR pred, envconsc_4|In welke mate ben je het eens of oneens met de volgende stellingen? - Iedereen zou zich moeten inspannen om onze natuurlijke hulpbronnen te behouden.|
|CDR pred, socconsc_1|In welke mate ben je het eens of oneens met de volgende stellingen? - Ik ben een sociaal bewust persoon.|
|CDR pred, socconsc_2|In welke mate ben je het eens of oneens met de volgende stellingen? - Ik ben betrokken bij het verbeteren van het welzijn van de samenleving.|
|CDR pred, socconsc_3|In welke mate ben je het eens of oneens met de volgende stellingen? - Ik streef hoge ethische normen na.|
|CDR pred, trust_1|In welke mate ben je het eens of oneens met de volgende stellingen over bedrijven? - Ik kan erop vertrouwen dat bedrijven mensen eerlijk en rechtvaardig behandelen.|
|CDR pred, trust_2|In welke mate ben je het eens of oneens met de volgende stellingen over bedrijven? - Ik kan erop vertrouwen dat bedrijven rekening houden met mensen zoals mezelf als ze een belangrijke beslissing nemen.|
|CDR pred, trust_3|In welke mate ben je het eens of oneens met de volgende stellingen over bedrijven? - Ik kan erop vertrouwen dat bedrijven hun beloften nakomen.|
|CDR pred, trust_4|In welke mate ben je het eens of oneens met de volgende stellingen over bedrijven? - Ik heb veel vertrouwen in de vaardigheden van bedrijven.|
|CDR pred, trust_5|In welke mate ben je het eens of oneens met de volgende stellingen over bedrijven? - Ik kan erop vertrouwen dat bedrijven competent zijn om te doen wat ze zeggen dat ze zullen doen.|

---

### ⤷ codebook_digiq_key.md




# Codebook: DigiQ
|Variabele|Volledige NL-vraag (QSF-verrijkt of CSV)|Opmerking/Key|
|:---------|:----------------------------------------|:-------------|
|CDR pred, digiQ_1|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - Iedereen krijgt dezelfde informatie als ze online naar dezelfde dingen zoeken.|Fout/Onjuist = Correct|
|CDR pred, digiQ_2|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - Voordat je een foto waarop een vriend duidelijk te zien is digitaal deelt, moet je altijd toestemming vragen.|Juist = Correct|
|CDR pred, digiQ_3|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - Sommige mensen worden betaald om producten te gebruiken in de video's die ze maken.|Juist = Correct|
|CDR pred, digiQ_4|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - Om je apparaten veiliger te houden, moet je updates altijd meteen installeren.|Juist = Correct|
|CDR pred, digiQ_5|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - Wat jij online doet, gebruiken bedrijven om hun producten en diensten te adverteren.|Juist = Correct|
|CDR pred, digiQ_6|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - Je slaapt slechter als je een smartphone of computer gebruikt vlak voor je naar bed gaat.|Juist = Correct|
|CDR pred, digiQ_7|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - In telefoons zitten stoffen die door mijnwerkers uit mijnen gehaald worden.|Juist = Correct|
|CDR pred, digiQ_8|De volgende uitspraken gaan over het internet. Geef aan of de zin volgens jou waar of niet waar is. Als je het niet weet, kies dan ‘Ik weet het niet’. Als je de vraag niet begrijpt, kies dan ‘Ik begrijp de vraag niet’. - Je online gedrag bepaalt wat je te zien krijgt op websites en apps voor nieuws en entertainment.|Juist = Correct|

**Scoring:** Correct=1; Incorrect=0; 'Ik weet het niet'=0; 'Ik begrijp de vraag niet'=NaN.

### 📊 Correlations (DataFrame view)

### 🔧 Predictor Reliability — Diagnostics (CSV view)